In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import needed libraries...
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#load the data set...
house=pd.read_csv("../input/bengaluru-house-price-data/Bengaluru_House_Data.csv")
house.head()

In [ ]:
#checking null values in percentage....
house.isnull().mean()*100

In [ ]:
#dropping society column , it is conatin more than 40% values are as null
house.drop(['society'],axis='columns',inplace=True)
#dropping na values because na values are present in very less ammount...
house.dropna(inplace=True)
house.shape

In [ ]:
pd.set_option('display.max_row',None)
pd.set_option('display.max_columns',None)
house['total_sqft'].unique().tolist()

In [ ]:
#taking only numeric values from size column and add them to new column bhk...
house['bhk']=house['size'].str.slice(0,1).astype(int)

In [ ]:
house['bhk'].head()

In [ ]:
#converting total_sqft column into numeric form ,some values in total_sqft be like
#'2100 - 2850','34.46Sq. Meter'
def convert_into_int(x):
    temp=x.split('-')
    if len(temp)==2:
        return (float(temp[0])+float(temp[-1]))/2
    try:
        return float(x)
    except:
        return None
house['total_sqft']=house['total_sqft'].apply(convert_into_int)

In [ ]:
house.dropna(inplace=True)
house=house.reset_index(drop=True)
house.head()

In [ ]:
#now visualizing the numericcal data to gain some information...
numeric_col=[feature for feature in house if house[feature].dtype!='O']
numeric_col

In [ ]:
house[numeric_col].describe()

In [ ]:
#visualizing correlaiton...
plt.figure(figsize=(10,8))
sns.heatmap(house[numeric_col].corr(),annot=True,cmap='coolwarm')
plt.title('Correlation of Numerical Data',fontsize=30)
plt.show()


In [ ]:
#seeing distribution of that data with the help of pairplot
plt.figure(figsize=(10,8))
sns.pairplot(house[numeric_col])
plt.show()

In [ ]:
#visualizing the distribution of every feature seperatily with the help of histogram...
plt.figure(figsize=(10,8))
for i,col in enumerate(numeric_col):
    plt.subplot(3,2,i+1)
    plt.hist(house[col],bins=30)
    plt.title(col)
plt.tight_layout(pad=1.0)
plt.show()
    

In [ ]:
#visualizing the outliers...
plt.figure(figsize=(12,10))
for i,col in enumerate(numeric_col):
    plt.subplot(3,2,i+1)
    sns.boxplot(y=house[col])
plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
#if bath values more than 7 and bhk values are more than 6 remove them...
house=house[house['bath']<7]
house=house[house['bhk']<6]
house.shape

In [ ]:
sns.boxplot(y=house['bath'])

In [ ]:
#adding new feature price_persqft to know more info about the outliers...
house['price_per_sqft']=house['price']*100000/house['total_sqft']
house.head()

In [ ]:
house['price_per_sqft'].describe()

In [ ]:
#removing outlier with the help of price_per_sqft feature....
#removing those data points which are belonging out side the 3 standard devaition....
def remove_outliers(data):
    new_data=pd.DataFrame()
    for key,subdf in data.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        #print(m)
        s=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-3*s)) & (subdf.price_per_sqft<=(m+3*s))]
        #print(reduced_df)
        new_data=pd.concat([new_data,reduced_df],ignore_index=True)
    return new_data
new_house=remove_outliers(house)
new_house.shape
        

In [ ]:
#removing those columns which are not needed...
new_house.drop(['size'],inplace=True,axis=1)

In [ ]:
#now working with categorical features ....
cat_data=[feat for feat in new_house if new_house[feat].dtype=='O']
cat_data

In [ ]:
#visualizing the categorical data unique values...
sns.barplot(new_house['area_type'].unique(),new_house['area_type'].value_counts())
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(new_house['location'].value_counts()[:20].keys(),new_house['location'].value_counts()[:20])
plt.xticks(rotation=40)
plt.show()

In [ ]:
#area_type has few values Carpet Area so removing those values where carpet area is present....
new_house=new_house[new_house['area_type']!='Carpet  Area']
new_house.shape

In [ ]:
#encoding of area_type feature...
for area in new_house['area_type']:
    new_house['area_type_'+area]=np.where(new_house['area_type']==area,1,0)

In [ ]:
#availability feature has all values are in time format except one Redy To Move ...
#considering only Ready To Move as 1 and all others are as 0
new_house['availability_enc']=np.where(new_house['availability']=='Ready To Move',1,0)

In [ ]:
new_house.tail()

In [ ]:
#cahecking location feature before encoding it...
location_counts=new_house['location'].value_counts(ascending=False)
location_counts

In [ ]:
#in location feature if value_counts greater than 20 taking only those values.  
#if value_counts less than 20 consering those as other...
less_than20=location_counts[location_counts<21]
less_than20

In [ ]:
#function for defining value as 'other' if value_counts less than 20 otherwise take values same as  
def other(x):
    if x in less_than20:
        return 'other'
    else:
        return x
new_house['location']=new_house['location'].apply(other)


In [ ]:
new_house['location'].value_counts()

In [ ]:
#dropping values where other is present...
new_house=new_house[new_house['location']!='other']
new_house.shape

In [ ]:
#encoding location feature...
for val in new_house['location']:
    new_house['location_'+val]=np.where(new_house['location']==val,1,0)

In [ ]:
#dropping categorical columns...
clean_data=new_house.drop(['area_type','availability','location'],axis=1)
clean_data.shape

In [ ]:
clean_data=clean_data.reset_index(drop=True)
clean_data.head()

In [ ]:
#visualizing distriibution of contnious features....
plt.figure(figsize=(12,8))
for i,feat in enumerate(clean_data.iloc[:,:6]):
    plt.subplot(3,2,i+1)
    sns.distplot(clean_data[feat],kde=False,bins=30)
    plt.tight_layout(pad=1)
plt.show()

# Machine Learning model building ...

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
#diving data into train and test...
X=clean_data.drop('price',axis=1)
y=clean_data.price

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
print("Shape of X_train = ",X_train.shape)
print("Shape of X_test = ",X_test.shape)
print("Shape of y_train = ",y_train.shape)
print("Shape of y_test = ",y_test.shape)

In [ ]:
rf_model=RandomForestRegressor()
rf_model.fit(X_train,y_train)

In [ ]:
crossval=cross_val_score(rf_model,X_test,y_test)
crossval.mean()

In [ ]:
y_pred=rf_model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_pred)